# Tutorial 12: Object Oriented Design and Symbolic Algebra

In this tutorial we will design an object oriented system that will provide functionality for evaluating algebraic expressions. We will have to make good object oriented design choices and take advantage of class inheritance in order to represent shared properties among our multiple types. Our code should create "expression objects" and provide methods for evaluating these expressions, taking derivatives as well as obtainng their string representation. So we will implement our own python algebraic calculations framework!  

We want to be able to construct expressions that contain variables, numbers and operators. The expressions we are going to care about can all be broken down into a bunch of binary expressions of the form

L op R

We can rewrite the following expression

8y * 7x^2 + 3

((8 * y) * (7 * (x * x))) + 3

The operations we want to do are similar for many types of expressions. We will use Classes and other Object Oriented Programming (OOP) principles to
   
   1. Reduce the amount of repeated code
   2. Make it easy to add new expressions

### Class Hierarchy

Our expressions will be made up of variables, numbers and binary operations. Each of this will be referred to as symbols. There will be functionality common to all of them. As a result, they will all extend a Expression interface that will maintain it

The Var class will represent variables

The Num class will represent numbers

The BinOp class will hold logic common to all expressions of the form L op R This includes  
&nbsp;&nbsp;&nbsp;&nbsp;Add (Adding)  
&nbsp;&nbsp;&nbsp;&nbsp;Sub (Subtract)  
&nbsp;&nbsp;&nbsp;&nbsp;Mul (Multiplication)  
&nbsp;&nbsp;&nbsp;&nbsp;Div (Division)  

The class hierarchy tree is below

Expression  
│─── Var  
│─── Num  
│─── BinOp  
&nbsp;&nbsp;&nbsp;&nbsp;│─── Add  
&nbsp;&nbsp;&nbsp;&nbsp;│─── Sub  
&nbsp;&nbsp;&nbsp;&nbsp;│─── Mul  
&nbsp;&nbsp;&nbsp;&nbsp;│─── Div  

Notice how we built our expression objects as a recursive nested data structure! You can view the Var and Num variants as "base cases", since these are the building blocks of each expression. Moreover, BinOp ( and its child classes ) can be interpreted as the "recursive case", since each of them is defined in terms of a "left" and a "right" expression, which might itself be a BinOp!

In [18]:
class Expression:
    """Interface for algebraic expressions"""
    
    def __init__(self):
        pass
    
    def __str__(self):
        """
        :return: string form of this expression
        as it is generated by print()
        """
        raise NotImplementedError("not implemented")
    
    def __repr__(self):
        """
        :return: string representation of this expression
        """
        raise NotImplementedError("not implemented")
    
    def deriv(self, name):
        """
        Computes the derivative of the given expression
        with respect to variable name
        :return: another Expression object representing
        the derivative of this expression
        """
        raise NotImplementedError("not implemented")
    
    def _eval(self, mapping):
        """
        Given a dictionary mapping variable names to numerical
        values, evaluates this expression.
        :return: a float/int, the result of evaluting this expression
        """
        raise NotImplementedError("not implemented")    

    def __add__(left, right):
        return Add(left, right)

    def __sub__(left, right):
        return Sub(left, right)

    def __mul__(left, right):
        return Mul(left, right)

    def __truediv__(left, right):
        return Div(left, right)

    def __radd__(left, right):
        return Add(left, right)

    def __rsub__(left, right):
        return Sub(left, right)

    def __rmul__(left, right):
        return Mul(left, right)

    def __rtruediv__(left, right):
        return Div(left, right)    

Let's start by looking at some examples of expressions that we would like to represent as objects:

3 - numbers

x - variables

x + 3, x - 3 - addition and subtraction

x * 3, x / 2 - division and multiplication

(x + 5) * ( y - 3) + 25 / 5 - more complex expressions!

It seems like the "building blocks", the units, of each expression arenumbers and variables. Moreover, the behavior of these two kinds of expressions is pretty different:

3' = 0 while x' = 1

3 evalautes to 3, while x evaluates to the value associated with this variable name in a provided mapping. For example, if mapping is:

`mapping = {"x": 10, "y": 5}`

then x evalautes to 10.

It seems like it would be a good idea to create a separate class for each of them. We will call these implementations of the Expression interface Var and Num.


In [19]:
class Var(Expression):
    def __init__(self, n):
        super(Var, self).__init__()
        """
        Initializer.  Store an instance variable called `name`, containing the
        value passed in to the initializer.
        """
        self.name = n

    def __str__(self):
        return self.name

    def __repr__(self):
        return 'Var(' + repr(self.name) + ')'

    def deriv(self, name):
        if self.name == name:
            return Num(1)
        else:
            return Num(0)
    
    def _eval(self, mapping):
        if self.name in mapping:
            return mapping[self.name]
        else:
            return self

class Num(Expression):
    def __init__(self, n):
        super(Num, self).__init__()
        """
        Initializer.  Store an instance variable called `n`, containing the
        value passed in to the initializer.
        """
        self.n = n

    def __str__(self):
        return str(self.n)

    def __repr__(self):
        return 'Num(' + repr(self.n) + ')'
    
    def deriv(self, value):
        return Num(0)

    def _eval(self, mapping):
        return self.n

We also need to support actual operations, like addition, subtraction, multiplication and division. What are some common features that all of these operations have? 

1) They all have a left and a right expression that can be added/subtracted/multiplied/devided. Notice how we can apply these operations on the smallest blocks (var and num), or on some other expressions.
		right, left: Expression, Var, Num

2) They are all represented ( in string form ) by a certain operator: +, -, /, *

So we have the opportunity to integrate a new class in our design, BinOp, which is another implementation of Expression and from which our other classes will inherit.

In [20]:
class BinOp(Expression):
    def __init__(self, left, right):
        super(BinOp, self).__init__()
        left_type = type(left)
        if isinstance(left, Expression): self.left = left
        elif left_type == str: self.left = Var(left)
        elif left_type == int:self.left = Num(left)
        else: raise TypeError(
                "left is of an invalid type {}".format(
                    left_type,
                )
            )
        right_type = type(right)
        if isinstance(right, Expression): self.right = right
        elif right_type == str: self.right = Var(right)
        elif right_type == int: self.right = Num(right)
        else: raise TypeError(
                "right is of an invalid type {}".format(
                    right_type,
                )
            )

        self._operator = None
        self._order = None
        self._special = None

    def __repr__(self):
        name = self.__class__.__name__
        return name + "(" + repr(self.left) + ", " + repr(self.right) + ")"

    def __str__(self):
        operand_string = " " + self._operator + " "
        if isinstance(self.left, BinOp):
            if self.left._order < self._order:
                left_string = "(" + str(self.left) + ")"
            else:
                left_string = str(self.left)
        else:
            left_string = str(self.left)
        if isinstance(self.right, BinOp):
            if self.right._order < self._order or (self.right._order == self._order and self._special):
                right_string = "(" + str(self.right) + ")"
            else:
                right_string = str(self.right)
        else:
            right_string = str(self.right)
        return left_string + operand_string + right_string
    
    def derive(self, name):
        pass
    
    def _eval(self, mapping):
        pass

Now that we enclosed all common features of arithmetic operations in the same class we can go ahead and implement the Add, Sub, Mul and Div classes which would represent specific arithmetic operations, and which contain implementations of the behavior specific to each of these.

In [21]:
class Add(BinOp):
    def __init__(self, left, right):
        BinOp.__init__(self, left, right)
        self._operator = '+'
        self._order = 0
        self._special = True

    def deriv(self, value):
        return Add(self.left.deriv(value), self.right.deriv(value))

    def _eval(self, mapping):
        return self.left._eval(mapping) + self.right._eval(mapping)

class Sub(BinOp):
    def __init__(self, left, right):
        BinOp.__init__(self, left, right)
        self._operator = '-'
        self._order = 1
        self._special = True
    
    def deriv(self, value):
        return Sub(self.left.deriv(value), self.right.deriv(value))

    def _eval(self, mapping):
        return self.left._eval(mapping) - self.right._eval(mapping)


class Mul(BinOp):
    def __init__(self, left, right):
        BinOp.__init__(self, left, right)
        self._operator = '*'
        self._order = 1
        self._special = False

    def deriv(self, value):
        left_deriv = self.left.deriv(value)
        right_deriv = self.right.deriv(value)
        return Add(Mul(self.left, right_deriv), Mul(left_deriv, self.right))

    def _eval(self, mapping):
        return self.left._eval(mapping) * self.right._eval(mapping)

class Div(BinOp):
    def __init__(self, left, right):
        BinOp.__init__(self, left, right)
        self._operator = '/'
        self._order = 1
        self._special = True
    
    def deriv(self, value):
        left_deriv = self.left.deriv(value)
        right_deriv = self.right.deriv(value)
        return Div(Sub(Mul(self.right, left_deriv), Mul(self.left, right_deriv)), Mul(self.right, self.right))

    def _eval(self, mapping):
        return self.left._eval(mapping) / self.right._eval(mapping)

Notice that instantiating an object for even a simple expression like (3 + x) * y can become tedious. Therefore, we would like touse the python arithmetic operators in order to build expressions faster. For example, instead of always having to write the folowing piece of code:

In [22]:
expr = Mul(Add(Num(3), Var("x")), Var("y"))
print(expr)
expr

(3 + x) * y


Mul(Add(Num(3), Var('x')), Var('y'))

We can overwrite some of the python built in operators by implementingthe following methods on the Expression interface: \_\_add\_\_, \_\_sub\_\_, \_\_mul\_\_, \_\_truediv\_\_, \_\_radd\_\_, \_\_rsub\_\_, \_\_rmul\_\_, \_\_rtruediv\_\_. This way, the following code would create the same expression object:

In [24]:
expr = (3 + Var("x")) * Var("y")
print(expr)

(x + 3) * y


It seems like these methods are somehow similar to the methods on the GraphFactory class in labs 6 and 7! We pass a left and a right input to each of these and they create new Expression objects.

In [44]:
x = Var('x')
y = Var('y')
z = Var('z')

In [50]:
expr = x * x + y + 430 * z

In [51]:
print (expr)

x * x + y + z * 430


In [49]:
expr.deriv('x')._eval({'x': 4, 'y': 3, 'z': 12})

-314.5